## U1 - Drive and Camera Test Utility
Author: George Gorospe, george.gorospe@nmaia.net\
Last Update: June 1, 2025\
References: this notebook is based on prior work by Derek Runberg from Sparkfun.

### About: This utility is used to test the setup of the AI Explorer's motor commands and camera.

In [ ]:
#import required system based packages
import os
import sys
import time

#import required Jupter Notebook widget packages
import ipywidgets.widgets as widgets
from IPython.display import display
import traitlets


from jetcam.utils import bgr8_to_jpeg
from jetcam.csi_camera import CSICamera


#import the Sphero RVR SDK 
from sphero_sdk import SpheroRvrObserver
from sphero_sdk import RawMotorModesEnum

In [ ]:
# Camera Setup

# Instantiate the camera
camera = CSICamera(width=224, height=224)
image = camera.read()
camera.running = True

#instantiate the camera and define the image widget
image_widget = widgets.Image(format='jpeg', width=300, height=300)

# Callback function for each time a new frame is collected
def update_image(change):
    image = change['new']
    # Process the frame and display it
    image_widget.value = bgr8_to_jpeg(image)

# Start the update event observation and display the GUI
camera.observe(update_image, names='value')
display(image_widget)


In [ ]:
#instantiate the RVR object and wake up the robot to be ready for receiveing commands
rvr = SpheroRvrObserver()
rvr.wake()

# create button widgets for each direction
button_layout = widgets.Layout(width='100px', height='80px', align_self='center')
stop_button = widgets.Button(description='stop', button_style='danger', layout=button_layout)
forward_button = widgets.Button(description='forward', layout=button_layout)
backward_button = widgets.Button(description='backward', layout=button_layout)
left_button = widgets.Button(description='left', layout=button_layout)
right_button = widgets.Button(description='right', layout=button_layout)

# display buttons
middle_box = widgets.HBox([left_button,image_widget, right_button], layout=widgets.Layout(align_self='center'))
controls_box = widgets.VBox([forward_button, middle_box, backward_button,stop_button])
display(controls_box)

#define button event handler functions



def stop(change):
    rvr.raw_motors(
        left_mode=RawMotorModesEnum.off.value,
        left_duty_cycle=0,  # Valid duty cycle range is 0-255
        right_mode=RawMotorModesEnum.off.value,
        right_duty_cycle=0  # Valid duty cycle range is 0-255
        )
        
    
def step_forward(change):
    print("fire")
    rvr.raw_motors(
        left_mode=RawMotorModesEnum.forward.value,
        left_duty_cycle=128,  # Valid duty cycle range is 0-255
        right_mode=RawMotorModesEnum.forward.value,
        right_duty_cycle=128  # Valid duty cycle range is 0-255
        )
    time.sleep(.5)
    rvr.raw_motors(
        left_mode=RawMotorModesEnum.forward.value,
        left_duty_cycle=0,  # Valid duty cycle range is 0-255
        right_mode=RawMotorModesEnum.forward.value,
        right_duty_cycle=0  # Valid duty cycle range is 0-255
        )

def step_backward(change):
    rvr.raw_motors(
        left_mode=RawMotorModesEnum.reverse.value,
        left_duty_cycle=128,  # Valid duty cycle range is 0-255
        right_mode=RawMotorModesEnum.reverse.value,
        right_duty_cycle=128  # Valid duty cycle range is 0-255
        )
    time.sleep(.25)
    rvr.raw_motors(
        left_mode=RawMotorModesEnum.forward.value,
        left_duty_cycle=0,  # Valid duty cycle range is 0-255
        right_mode=RawMotorModesEnum.forward.value,
        right_duty_cycle=0  # Valid duty cycle range is 0-255
        )

def step_left(change):
    rvr.raw_motors(
        left_mode=RawMotorModesEnum.reverse.value,
        left_duty_cycle=128,  # Valid duty cycle range is 0-255
        right_mode=RawMotorModesEnum.forward.value,
        right_duty_cycle=128  # Valid duty cycle range is 0-255
        )
    time.sleep(.25)
    rvr.raw_motors(
        left_mode=RawMotorModesEnum.off.value,
        left_duty_cycle=0,  # Valid duty cycle range is 0-255
        right_mode=RawMotorModesEnum.off.value,
        right_duty_cycle=0  # Valid duty cycle range is 0-255
        )

def step_right(change):
    rvr.raw_motors(
        left_mode=RawMotorModesEnum.forward.value,
        left_duty_cycle=128,  # Valid duty cycle range is 0-255
        right_mode=RawMotorModesEnum.reverse.value,
        right_duty_cycle=128  # Valid duty cycle range is 0-255
    )
    time.sleep(.25)
    rvr.raw_motors(
        left_mode=RawMotorModesEnum.off.value,
        left_duty_cycle=0,  # Valid duty cycle range is 0-255
        right_mode=RawMotorModesEnum.off.value,
        right_duty_cycle=0  # Valid duty cycle range is 0-255
        )

#attach the event handler functions to specific button events
forward_button.on_click(step_forward)
backward_button.on_click(step_backward)
left_button.on_click(step_left)
right_button.on_click(step_right)
stop_button.on_click(stop)